# ※ <한강홍수통제소_OPEN API를 이용한 수위 데이터 수집> 따라하기
- 한강홍수통제소 OPEN API
> http://www.hrfco.go.kr/web/openapiPage/openApi.do

In [1]:
SERVICE_KEY = "E49E53F1-C36B-4EB5-B8F6-F0C91AC036F0"

In [2]:
import pandas as pd
import numpy as np
import requests
import calendar
import time

In [3]:
# 수위 관측소 제원 요청
url = f'http://api.hrfco.go.kr/{SERVICE_KEY}/waterlevel/info.json'
response = requests.get(url)

data = response.json()

In [5]:
# 수위 관측소 필터링
for con in data['content']:
    if '서울' in con['obsnm']:
        print(con['obsnm'], con['wlobscd'])
    if '팔당'  in con['obsnm']:
        print(con['obsnm'], con['wlobscd'])

팔당댐 1017678
남양주시(팔당댐) 1017690
남양주시(팔당대교) 1018610
팔당대교 1018611
서울시(광진교) 1018640
서울시(대곡교) 1018655
서울시(대치교) 1018658
서울시(청담대교) 1018662
서울시(창동교) 1018669
서울시(월계2교) 1018670
서울시(중랑교) 1018675
서울시(잠수교) 1018680
서울시(한강대교) 1018683
서울시(너부대교) 1018695
서울시(오금교) 1018697
서울시(행주대교) 1019630


In [23]:
bridge = {
    # "남양주시(팔당대교)": 1018610,
    # "서울시(광진교)": 1018640,
    # '팔당댐': 1017678,
    # '남양주시(팔당댐)': 1017690,
    # '팔당대교': 1018611,
    # '서울시(대곡교)': 1018655,
    # '서울시(대치교)': 1018658,
    '서울시(청담대교)': 1018662,
    '서울시(창동교)': 1018669,
    '서울시(월계2교)': 1018670,
    '서울시(중랑교)': 1018675,
    '서울시(잠수교)': 1018680,
    '서울시(한강대교)': 1018683,
    '서울시(너부대교)': 1018695,
    '서울시(오금교)': 1018697,
    '서울시(행주대교)': 1019630
}

In [24]:
# bridge 변수에 저장한 다리의 수위 자료 요청
for name, code in bridge.items():
    for year in range(2012, 2023):
        ms, me = (5, 11) if year != 2022 else (5, 8)
        for month in range(ms, me):
            weekday, end = calendar.monthrange(year, month)
            sdate = f"{year}{month:02}010000" 
            edate = f"{year}{month:02}{end:02}2350"
            url = f"http://api.hrfco.go.kr/{SERVICE_KEY}/waterlevel/list/10M/{code}/{sdate}/{edate}.json"            
            response = requests.get(url)
            df = pd.DataFrame(response.json()['content'])
            df.to_csv(f"한강홍수통제소/{name}/{year}{month:02}_{name}.csv")
            time.sleep(3)

# ※ csv 파일 불러오기
- [파이썬 기초] 특정 폴더에 있는 파일명 추출 하는법
> https://bit.ly/3KkYwfp
- [파이썬] Python에서 특정 문자열 포함여부 확인하는 방법(실무에서 많이 사용합니다.)
> https://bit.ly/3AM4BhW

In [9]:
from glob import glob
import os
import warnings        
warnings.filterwarnings("ignore") 

In [25]:
# '한강홍수통제소' 라는 폴더명을 가진 디렉토리로 이동
os.chdir("./한강홍수통제소")

# 해당 폴더 내에 어떤 파일들이 있는지 확인
print(os.listdir())

['남양주시(팔당대교)', '남양주시(팔당댐)', '서울시(광진교)', '서울시(너부대교)', '서울시(대곡교)', '서울시(대치교)', '서울시(오금교)', '서울시(월계2교)', '서울시(잠수교)', '서울시(중랑교)', '서울시(창동교)', '서울시(청담대교)', '서울시(한강대교)', '서울시(행주대교)', '팔당대교', '팔당댐']


In [26]:
folder_list=os.listdir()

In [30]:
from glob import glob

In [79]:
len(folder_list)

16

In [88]:
tmp=pd.DataFrame()
for i in range(0, len(folder_list)):
    if i==0:
        file_list = sorted(glob(folder_list[i]+"/*.csv"))
        for j in file_list:
            df=pd.read_csv(j)
            df=df[['ymdhm','wl','fw']]
            tmp=pd.concat([tmp, df])
            # print(tmp.shape)
        tmp.rename(columns={'wl':j.split('\\')[0]+'_wl','fw':j.split('\\')[0]+'_fw'}, inplace=True)
    else:
        file_list = sorted(glob(folder_list[i]+"/*.csv"))
        tmp2=pd.DataFrame()
        for j in file_list:
            df=pd.read_csv(j)
            df=df[['ymdhm','wl','fw']]
            tmp2=pd.concat([tmp2,df])
        tmp2=tmp2.rename(columns={'wl':j.split('\\')[0]+'_wl','fw':j.split('\\')[0]+'_fw'})
        tmp=pd.merge(tmp, tmp2, how='left', on='ymdhm')

In [89]:
tmp

,ymdhm,남양주시(팔당대교)_wl,남양주시(팔당대교)_fw,남양주시(팔당댐)_wl,남양주시(팔당댐)_fw,서울시(광진교)_wl,서울시(광진교)_fw,서울시(너부대교)_wl,서울시(너부대교)_fw,서울시(대곡교)_wl,...,서울시(청담대교)_wl,서울시(청담대교)_fw,서울시(한강대교)_wl,서울시(한강대교)_fw,서울시(행주대교)_wl,서울시(행주대교)_fw,팔당대교_wl,팔당대교_fw,팔당댐_wl,팔당댐_fw
0,201205312350,1.3,295.66,25.03,0.0,1.14,2054.51,0.12,0.76,1.75,...,1.19,355.94,0.69,370.24,1.83,406.79,,,,
1,201205312340,1.3,295.66,25.04,0.0,1.14,2054.51,0.12,0.76,1.75,...,1.19,355.94,0.69,411.57,1.83,406.79,,,,
2,201205312330,1.3,295.66,25.06,0.0,1.13,2054.51,0.12,0.76,1.75,...,1.19,355.94,0.69,404.96,1.83,406.79,,,,
3,201205312320,1.3,295.66,25.06,0.0,1.12,2054.51,0.12,0.76,1.75,...,1.22,374.74,0.69,438.02,1.83,406.79,,,,
4,201205312310,1.3,295.66,25.06,0.0,1.12,2054.51,0.12,0.76,1.75,...,1.22,374.74,0.69,424.79,1.83,406.79,,,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
278203,202207010040,5.97,9713.77,24.54,,4.32,12268.0,1.07,15.0,3.34,...,7.18,,5.46,12099.28,5.22,11820.61,,,,
278204,202207010030,5.99,9768.38,24.54,,4.4,12806.18,1.1,15.67,3.38,...,7.22,,5.4,12324.36,5.23,11878.19,,,,
278205,202207010020,6.02,9850.49,24.55,,4.39,12520.77,1.13,16.36,3.35,...,7.26,,5.44,12252.34,5.24,11935.91,,,,
278206,202207010010,6.01,9823.1,24.55,,4.43,12841.98,1.16,17.05,3.40,...,7.29,,5.55,12712.39,5.25,11993.78,,,,


In [90]:
tmp.shape

(278208, 33)

In [93]:
os.chdir("../")

In [94]:
tmp.to_csv('dam_api.csv',index=False)